In [2]:
import pandas as pd
import numpy as np

In [95]:
data = pd.read_csv("data_diamantes.csv") 

In [96]:
test = pd.read_csv("test_diamantes.csv") 

In [97]:
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,2.26,Ideal,G,SI2,61.9,57.0,8.44,8.36,5.20,12831
1,2.43,Very Good,H,SI2,63.2,57.0,8.56,8.50,5.39,16170
2,0.80,Premium,F,SI2,61.0,57.0,6.03,6.01,3.67,2797
3,0.40,Ideal,F,I1,63.3,60.0,4.68,4.64,2.95,630
4,0.31,Ideal,G,VS2,61.6,55.0,4.39,4.37,2.70,698


In [98]:
data.cut.value_counts()

Ideal        16139
Premium      10377
Very Good     9101
Good          3650
Fair          1188
Name: cut, dtype: int64

In [99]:
data.price.mean()

3939.2428130021012

In [100]:
data.isnull().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
price      0
dtype: int64

In [101]:
test = test [["id", "carat", "cut", "color", "x", "y", "z"]]

In [102]:
data = data[["carat", "cut", "color", "x", "y", "z", "price"]]

In [103]:
data

,carat,cut,color,x,y,z,price
0,2.26,Ideal,G,8.44,8.36,5.20,12831
1,2.43,Very Good,H,8.56,8.50,5.39,16170
2,0.80,Premium,F,6.03,6.01,3.67,2797
3,0.40,Ideal,F,4.68,4.64,2.95,630
4,0.31,Ideal,G,4.39,4.37,2.70,698
...,...,...,...,...,...,...,...
40450,1.11,Premium,H,6.63,6.56,4.14,5315
40451,0.73,Ideal,F,5.77,5.74,3.60,2762
40452,1.26,Very Good,I,7.09,7.02,4.18,6855
40453,0.72,Ideal,G,5.76,5.83,3.56,2297


In [104]:
def cleanColor(color):
    if color == "D":
        return 1
    if color == "E":
        return 1
    if color == "F":
        return 2
    if color == "G":
        return 2
    if color == "H":
        return 3
    if color == "I":
        return 3
    else:
        return 4
    
data["color"]= data["color"].apply(cleanColor)
test["color"]= test["color"].apply(cleanColor)

In [105]:
data.cut.value_counts()

Ideal        16139
Premium      10377
Very Good     9101
Good          3650
Fair          1188
Name: cut, dtype: int64

In [106]:
X = pd.get_dummies(data,columns=["cut"])
X_submission = pd.get_dummies(test,columns=["cut"])
y = X['price']
submission_Id = test.id


In [107]:
X = X.drop(columns=['price'],axis=1)
X_submission = X_submission.drop(columns=['id'],axis=1)

In [108]:
X["x"] = X["x"]/10
X["y"] = X["y"]/10
X["z"] = X["z"]/10

In [109]:
X_submission["x"] = X_submission["x"]/10
X_submission["y"] = X_submission["y"]/10
X_submission["z"] = X_submission["z"]/10

In [110]:
X

,carat,color,x,y,z,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good
0,2.26,2,0.844,0.836,0.520,0,0,1,0,0
1,2.43,3,0.856,0.850,0.539,0,0,0,0,1
2,0.80,2,0.603,0.601,0.367,0,0,0,1,0
3,0.40,2,0.468,0.464,0.295,0,0,1,0,0
4,0.31,2,0.439,0.437,0.270,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
40450,1.11,3,0.663,0.656,0.414,0,0,0,1,0
40451,0.73,2,0.577,0.574,0.360,0,0,1,0,0
40452,1.26,3,0.709,0.702,0.418,0,0,0,0,1
40453,0.72,2,0.576,0.583,0.356,0,0,1,0,0


In [111]:
X["carat"] = X["carat"]*1.5
X_submission["carat"] = X_submission["carat"]*1.5

In [127]:
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
models = {
 #   "modelSVR": SVR(kernel="linear"),
 #  "modelLinearRG": LinearRegression(),
   "decisionTree": DecisionTreeRegressor(random_state=0)
 #   'randomForest': RandomForestRegressor(max_depth=3, random_state=0,n_estimators=50)
}

preds = {}

for label, model in models.items():
    print("Training {}".format(label))
    model.fit(X,y)
    preds[label] = model.predict(X_submission)

Training decisionTree


In [128]:
for label, pred in preds.items():
    submission = pd.DataFrame({
        'Id': submission_Id,
        'price': pred
    })
    display(submission.head())

,Id,price
0,0,2195.0
1,1,10984.0
2,2,1746.0
3,3,2795.0
4,4,1753.0


In [129]:
submission.price.mean()

3934.0310076451788

In [130]:
submission.to_csv('submission_tree.csv', index=False)